In [ ]:
# -*- coding: utf-8 -*-
"""과제5 CNN기반 영상분류문제

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1Py2GvZc1GwqNbrR4fuQrHFy6zBEdF1xF
"""

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense
from tensorflow.keras.optimizers import Adam
import os
from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import train_test_split

In [ ]:

(x_train,y_train),(x_test,y_test)=cifar10.load_data()
x_train=x_train.astype(np.float32)/255.0
y_train=tf.keras.utils.to_categorical(y_train,10)

x_test=x_test.astype(np.float32)/255.0
y_test=tf.keras.utils.to_categorical(y_test,10)

#x_train, _, y_train,_ = train_test_split(x_train, y_train, test_size=0.5, random_state=42)
x_val, _, y_val,_ = train_test_split(x_test, y_test, test_size=0.8, random_state=42)

input_shape = x_train.shape[1:]

#동일조건 유지해야 하는 변수(두 모델 모두 동일하게 적용해야 함)
g_epoch = 80
g_batch = 128
np.random.seed(42)
tf.random.set_seed(42)

print("reduced train/val size:", len(x_train), len(x_val), "input shape:", input_shape)

170508288/170498071 [==============================] - 2s 0us/step
reduced train/val size: 50000 2000 input shape: (32, 32, 3)


In [ ]:
# to make this notebook's output stable across runs
import sys

print(sys.version)
tf.__version__

3.7.13 (default, Mar 16 2022, 17:37:17) 
[GCC 7.5.0]


'2.8.0'

In [ ]:
from tensorflow.keras.layers import MaxPooling2D, Dropout, Conv2D

cnn=Sequential()
cnn.add(Conv2D(64,(3,3),activation='relu',input_shape=(32,32,3)))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))
cnn.add(Conv2D(128,(3,3),activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))
cnn.add(Conv2D(256,(3,3),activation='relu', padding='same'))
cnn.add(Conv2D(256,(3,3),activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Flatten())
cnn.add(Dense(1000,activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(10,activation='softmax'))

cnn.compile(loss='categorical_crossentropy',optimizer=Adam(0.00002),metrics=['accuracy'])
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 15, 15, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 15, 15, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 128)        0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 128)         0

In [ ]:
  hist=cnn.fit(x_train, y_train, batch_size=g_batch, epochs=g_epoch,
             validation_data=(x_val,y_val), verbose=1)

g_org_res=cnn.evaluate(x_test,y_test,verbose=0)
print("Baseline 정확률은",g_org_res[1]*100)

Epoch 1/80
391/391 [==============================] - 16s 16ms/step - loss: 2.1611 - accuracy: 0.1947 - val_loss: 1.9965 - val_accuracy: 0.3070
Epoch 2/80
391/391 [==============================] - 6s 15ms/step - loss: 1.8852 - accuracy: 0.3166 - val_loss: 1.9138 - val_accuracy: 0.3070
Epoch 3/80
391/391 [==============================] - 6s 15ms/step - loss: 1.7594 - accuracy: 0.3623 - val_loss: 1.7949 - val_accuracy: 0.3515
Epoch 4/80
391/391 [==============================] - 6s 16ms/step - loss: 1.6829 - accuracy: 0.3900 - val_loss: 1.7792 - val_accuracy: 0.3695
Epoch 5/80
391/391 [==============================] - 6s 15ms/step - loss: 1.6187 - accuracy: 0.4137 - val_loss: 1.6647 - val_accuracy: 0.4055
Epoch 6/80
391/391 [==============================] - 6s 16ms/step - loss: 1.5685 - accuracy: 0.4312 - val_loss: 1.7197 - val_accuracy: 0.3900
Epoch 7/80
391/391 [==============================] - 6s 15ms/step - loss: 1.5259 - accuracy: 0.4479 - val_loss: 1.5950 - val_accuracy: 0.431

In [ ]:
no_class = 10

# for transfer learning only
from tensorflow.keras.applications import VGG16
he = tf.keras.initializers.HeNormal()

# for transfer learning only
transfermodel = VGG16(weights='imagenet',include_top=False,
                    input_shape=input_shape)
#base_model.trainable=False     # it's up to you
# your model architecture
model=Sequential()
model.add(transfermodel)    # for transfer learning only
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.BatchNormalization())
# model.add(Flatten())        # for transfer learning only
model.add(Dense(128,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(no_class, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer=Adam(0.00002),metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 global_average_pooling2d_1   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                                 
                                                                 
 dense_5 (Dense)             (None, 128)               65664     
                                                                 
 dense_6 (Dense)             (None, 32)                4128      
                                                                 
 dense_7 (Dense)             (None, 10)               

In [ ]:
hist=model.fit(x_train, y_train, batch_size=g_batch, epochs=g_epoch,
             validation_data=(x_val,y_val), verbose=1)

yours=model.evaluate(x_test,y_test,verbose=0)
print("Baseline vs yours: ",g_org_res[1]*100, yours[1]*100)

Epoch 1/80
391/391 [==============================] - 23s 56ms/step - loss: 1.2176 - accuracy: 0.6050 - val_loss: 0.8878 - val_accuracy: 0.7080
Epoch 2/80
391/391 [==============================] - 22s 57ms/step - loss: 0.6682 - accuracy: 0.7808 - val_loss: 0.7178 - val_accuracy: 0.7620
Epoch 3/80
391/391 [==============================] - 22s 57ms/step - loss: 0.5053 - accuracy: 0.8328 - val_loss: 0.6220 - val_accuracy: 0.7955
Epoch 4/80
391/391 [==============================] - 23s 58ms/step - loss: 0.3961 - accuracy: 0.8703 - val_loss: 0.6146 - val_accuracy: 0.8085
Epoch 5/80
391/391 [==============================] - 23s 58ms/step - loss: 0.3072 - accuracy: 0.8994 - val_loss: 0.6332 - val_accuracy: 0.7995
Epoch 6/80
391/391 [==============================] - 23s 59ms/step - loss: 0.2327 - accuracy: 0.9246 - val_loss: 0.6013 - val_accuracy: 0.8225
Epoch 7/80
391/391 [==============================] - 23s 59ms/step - loss: 0.1631 - accuracy: 0.9495 - val_loss: 0.5780 - val_accuracy: